# Based on http://pele-python.github.io/pele/tutorial_potential.html#finding-pathways-tutorial

## Creating Your Potential

pele has a number of potentials built in, including the Lennard-Jones potential which is used for most of our examples. It also has an interface to the GMIN program with which you can access any myriad of GMIN’s built in potentials. We also plan to build interfaces to some of the standard molecular dynamics packages like gromacs and OpenMM.

The flexibility of pele alows it to be easily used with just about any scalar function. Probably the biggest limitation is that, because we use a lot of gradient based local minimization you may experience problems if your potential is discontinuous.

## simple 1D potential

Lets create an artificial potential from the sum of a parabola and a cosine term:

In [13]:
from pele.potentials import BasePotential
import numpy as np
class My1DPot(BasePotential):
    def getEnergy(x):
        return np.cos(14.5 * x[0] - 0.3) + (x[0] + 0.2) * x[0]


The above definition of member function getEnergy() is all that is required to use the global optimization features of pele. It is derived from BasePotential, which will calculate gradient numerically. However, Defining an analytical gradient will make things run a lot faster:

In [14]:
from pele.potentials import BasePotential
import numpy as np
class My1DPot(BasePotential):
    """1d potential"""
    def getEnergy(self, x):
        return np.cos(14.5 * x[0] - 0.3) + (x[0] + 0.2) * x[0]

    def getEnergyGradient(self, x):
        E = self.getEnergy(x)
        grad = np.array([-14.5 * np.sin(14.5 * x[0] - 0.3) + 2. * x[0] + 0.2])
        return E, grad

From this point you can jump in and use BasinHopping to find the global minimum. The best way to do this is to use the convience wrapper, the system class. As a first start, all we must do is tell the system class what our potential is

In [15]:
from pele.systems import BaseSystem
class My1DSystem(BaseSystem):
    def get_potential(self):
        return My1DPot()

The following code will use the system class to initialize a basinhopping class and run basinhopping for 100 steps. We use an sqlite database to store the minima found:

In [18]:
system = My1DSystem()
database = system.create_database()
x0 = np.array([1.])
bh = system.get_basinhopping(database=database, coords=x0)
bh.run(100)
print("found", len(database.minima()), "minima")
min0 = database.minima()[0]
print("lowest minimum found at", min0.coords, "with energy", min0.energy)

will compute the lowest eigenvector by diagonalizing the Hessian
Qu   0 E= 0.42591968109168876 quench_steps= 6 RMS= 1.2346449973499318e-09 Markov E= 1.1372082770759182 accepted= True
Qu   1 E= -0.42166279553958996 quench_steps= 6 RMS= 7.647191456683355e-06 Markov E= 0.42591968109168876 accepted= True
Qu   2 E= -0.4216627955397285 quench_steps= 6 RMS= 6.274860075983213e-10 Markov E= -0.42166279553958996 accepted= True
Qu   3 E= -0.8972667196907541 quench_steps= 6 RMS= 3.4170399842992083e-10 Markov E= -0.4216627955397285 accepted= True
Qu   4 E= -0.8972667196907355 quench_steps= 6 RMS= 2.8086741850619212e-06 Markov E= -0.8972667196907541 accepted= True
Qu   5 E= -0.897266719690754 quench_steps= 7 RMS= 7.61706031582321e-10 Markov E= -0.8972667196907355 accepted= True
Qu   6 E= -0.8972667196907081 quench_steps= 6 RMS= 4.410721827219355e-06 Markov E= -0.897266719690754 accepted= True
Qu   7 E= -1.0008761844426532 quench_steps= 6 RMS= 9.795225759112736e-07 Markov E= -0.8972667196907081 accep